<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#train-on-BERT" data-toc-modified-id="train-on-BERT-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>train on BERT</a></span><ul class="toc-item"><li><span><a href="#初始化" data-toc-modified-id="初始化-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>初始化</a></span></li><li><span><a href="#HotpotQA_Dataset" data-toc-modified-id="HotpotQA_Dataset-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>HotpotQA_Dataset</a></span></li></ul></li><li><span><a href="#训练" data-toc-modified-id="训练-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>训练</a></span><ul class="toc-item"><li><span><a href="#辅助函数" data-toc-modified-id="辅助函数-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>辅助函数</a></span></li><li><span><a href="#超参数" data-toc-modified-id="超参数-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>超参数</a></span></li><li><span><a href="#参数配置" data-toc-modified-id="参数配置-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>参数配置</a></span></li><li><span><a href="#训练流程" data-toc-modified-id="训练流程-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>训练流程</a></span><ul class="toc-item"><li><span><a href="#训练辅助函数" data-toc-modified-id="训练辅助函数-2.4.1"><span class="toc-item-num">2.4.1&nbsp;&nbsp;</span>训练辅助函数</a></span></li><li><span><a href="#实例化" data-toc-modified-id="实例化-2.4.2"><span class="toc-item-num">2.4.2&nbsp;&nbsp;</span>实例化</a></span></li><li><span><a href="#训练" data-toc-modified-id="训练-2.4.3"><span class="toc-item-num">2.4.3&nbsp;&nbsp;</span>训练</a></span></li></ul></li></ul></li><li><span><a href="#草稿" data-toc-modified-id="草稿-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>草稿</a></span></li></ul></div>

# train on BERT

In [1]:
print_cpu_info = True
if print_cpu_info:
    r_OS_info = !cat /etc/issue
    r_cpu_type = !cat /proc/cpuinfo | grep name | cut -f2 -d: | uniq -c
    r_cpu_kernel = !cat /proc/cpuinfo | grep physical | uniq -c
    r_cpu_run_mode = !getconf LONG_BIT
    r_cpu_support_mode = !cat /proc/cpuinfo | grep flags | grep ' lm ' | wc -l

    OS_info = '\n'.join(r_OS_info)
    cpu_type = '\n'.join(r_cpu_type)
    cpu_kernel = '\n'.join(r_cpu_kernel)
    cpu_run_mode = '\n'.join(r_cpu_run_mode)
    cpu_support_mode = '\n'.join(r_cpu_support_mode)

    print(f'OS_info: {OS_info}')
    print(f'cpu_type: {cpu_type}')
    print(f'cpu_kernel: {cpu_kernel}')
    print(f'cpu_run_mode: {cpu_run_mode}')
    print(f'cpu_support_mode: {cpu_support_mode}')

OS_info: \S
Kernel \r on an \m

cpu_type:      20  Intel(R) Core(TM) i9-9820X CPU @ 3.30GHz
cpu_kernel:       1 physical id	: 0
      1 address sizes	: 46 bits physical, 48 bits virtual
      1 physical id	: 0
      1 address sizes	: 46 bits physical, 48 bits virtual
      1 physical id	: 0
      1 address sizes	: 46 bits physical, 48 bits virtual
      1 physical id	: 0
      1 address sizes	: 46 bits physical, 48 bits virtual
      1 physical id	: 0
      1 address sizes	: 46 bits physical, 48 bits virtual
      1 physical id	: 0
      1 address sizes	: 46 bits physical, 48 bits virtual
      1 physical id	: 0
      1 address sizes	: 46 bits physical, 48 bits virtual
      1 physical id	: 0
      1 address sizes	: 46 bits physical, 48 bits virtual
      1 physical id	: 0
      1 address sizes	: 46 bits physical, 48 bits virtual
      1 physical id	: 0
      1 address sizes	: 46 bits physical, 48 bits virtual
      1 physical id	: 0
      1 address sizes	: 46 bits physical, 48 bits vi

In [2]:
print_gpu_info = True
if print_gpu_info:
    gpu_info = !nvidia-smi
    gpu_info = '\n'.join(gpu_info)
    if gpu_info.find('failed') >= 0:
        print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
        print('and then re-execute this cell.')
    else:
        print(gpu_info)

Sun Mar  8 11:33:37 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.36       Driver Version: 440.36       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:19:00.0 Off |                  N/A |
| 27%   28C    P8     1W / 250W |      1MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:1A:00.0 Off |                  N/A |
| 27%   32C    P8     3W / 250W |      1MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G

In [3]:
print_ram_info = True
if print_ram_info:
    from psutil import virtual_memory
    ram_gb = virtual_memory().total / 1e9
    print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

    if ram_gb < 20:
        print('To enable a high-RAM runtime, select the Runtime → "Change runtime type"')
        print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
        print('re-execute this cell.')
    else:
        print('You are using a high-RAM runtime!')

Your runtime has 33.2 gigabytes of available RAM

You are using a high-RAM runtime!


## 初始化

In [4]:
try:
    from google.colab import drive
    drive.mount('/content/folders/')
    !pip install transformers
    # !pip install -U spacy[cuda100]
    # !wget -P /content/folders/My\ Drive/download/ https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.2.5/en_core_web_lg-2.2.5.tar.gz
    # !pip install /content/folders/My\ Drive/download/en_core_web_lg-2.2.5.tar.gz
    # !wget -P /content/folders/My\ Drive/HotpotQA/ http://curtis.ml.cmu.edu/datasets/hotpot/hotpot_train_v1.1.json
    # json_train_path = '/content/folders/My Drive/HotpotQA/样例_hotpot_train_v1.1.json' # 5个例子
    json_train_path = '/content/folders/My Drive/HotpotQA/hotpot_train_v1.1.json'
    save_cache_path = '/content/folders/My Drive/save_cache/'
    save_cache_path_linux = '/content/folders/My\ Drive/save_cache/'
    HotpotQA_path = '/content/folders/My Drive/HotpotQA'
except:
    json_train_path = r'./data/hotpot_train_v1.1.json'
    HotpotQA_path = './'
    save_cache_path = 'save_cache/'
    use_proxy = False
    proxies={"http_proxy": "127.0.0.1:10802",
        "https_proxy": "127.0.0.1:10802"} if use_proxy else None


In [5]:
from argparse import Namespace
import os
import json
import pickle

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm_notebook

from apex import amp

from transformers import AutoTokenizer, AutoModelForSequenceClassification

import sys
sys.path.insert(0,HotpotQA_path)

from Classes import Node, Question_Paragraph, Adjacency_sp

## HotpotQA_Dataset

In [6]:
class HotpotQA_Dataset(Dataset):
    '''index只要获取一条一条的(q,p,label)即可.'''
    def __init__(self, train_pair, val_pair, tokenizer):
        
        # features
        self.train_pair = train_pair # [(sen1, sen2, label), ...]
        self.train_size = len(self.train_pair)
        
        self.val_pair = val_pair
        self.val_size = len(self.val_pair)

        self._tokenizer = tokenizer
        
        # func
        self._lookup_dict = {'train': (self.train_pair, self.train_size),
                    'val': (self.val_pair, self.val_size)}

        self.pair_type=None
        self.set_split('train')

        # parameters
        self.max_length = 1024
        self.pad_to_max_length = True
        
    @classmethod
    def build_dataset(cls, hotpotQA_train_preprocess, pair_type='ques_sent', \
                      ratio_train=0.7, contain_title=False, \
                      tokenizer=None, seed=123):
        '''Q_P_list是由Question-Paragraph对组成的list.
        可以打平抽样. 否则难以提升batchsize.
        '''
        assert pair_type in ['ques_sent', 'ques_para']

        _Q_P_list = [i['ques_para_list'] for i in hotpotQA_train_preprocess]
        Q_P_list = []
        for i in _Q_P_list: Q_P_list.extend(i)

        np.random.seed(seed)
        np.random.shuffle(Q_P_list)

        sample_train_num = int(ratio_train * len(Q_P_list))

        if pair_type == 'ques_sent':
            train_pair = []
            for Q_P in Q_P_list[:sample_train_num]:
                train_pair.extend(Q_P.get_ques_sent_label_list())

            val_pair = []
            for Q_P in Q_P_list[sample_train_num:]:
                val_pair.extend(Q_P.get_ques_sent_label_list())       
        else:
            train_pair = [Q_P.get_ques_para_label_tuple(contain_title) \
                          for Q_P in Q_P_list[:sample_train_num]]
            val_pair = [Q_P.get_ques_para_label_tuple(contain_title) \
                        for Q_P in Q_P_list[sample_train_num:]]

        _cls = cls(train_pair, val_pair, tokenizer)
        _cls.pair_type = pair_type
        return _cls

    @staticmethod
    def build_dataset_from_path(QA_preprocess_path, pair_type='ques_sent', \
                      ratio_train=0.7, contain_title=False, \
                      tokenizer=None, seed=123):
        
        with open(QA_preprocess_path,'rb')as fp:
            hotpotQA_train_preprocess = pickle.load(fp)
        
        return HotpotQA_Dataset.build_dataset(hotpotQA_train_preprocess,
                            pair_type,
                            ratio_train,
                            contain_title,
                            tokenizer,
                            seed,)

    def set_parameters(self, max_length, pad_to_max_length):
        self.max_length = max_length
        self.pad_to_max_length = pad_to_max_length

    def set_split(self, split="train"):
        assert split in ['train', 'val', 'test']
        self._target_split = split
        self._target_pair, self._target_size = self._lookup_dict[split]

    def get_one_item(self, index):
        one_line = self.train_pair[index]
        return {'ques_tokens': one_line[0],
                'pair_tokens': one_line[1],
                'label': one_line[2]}

    def get_tokenizer(self):
        return self._tokenizer

    def __len__(self):
        return self._target_size

    def __getitem__(self, index):
        one_line = self.train_pair[index]

        model_input = self._tokenizer.encode_plus(
                text = one_line[0],
                text_pair = one_line[1],
                add_special_tokens = True,
                max_length = self.max_length,
                truncation_strategy = 'only_second',
                pad_to_max_length = self.pad_to_max_length,
                return_tensors = 'pt'
            )
        label = one_line[2]
        model_input['labels'] = torch.tensor([label], dtype=torch.long)

        return model_input
    
    def __repr__(self):
        return 'HotpotQA Dataset. mode: {}. pair: {}. size: {}. max_seq: {}'.format\
            (self._target_split,self.pair_type,self.__len__(),self.max_length)

    def __str__(self):
        return self.__repr__()

    def get_num_batches(self, batch_size):
        return len(self) // batch_size

def generate_batches(dataset, batch_size, shuffle=True, drop_last=True, device='cpu'): 
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size,
                    shuffle=shuffle, drop_last=drop_last)

    for data_dict in dataloader:
        out_data_dict = {}
        for name, _tensor in data_dict.items():
            out_data_dict[name] = data_dict[name].squeeze(1).to(device)
        yield out_data_dict

# 训练

## 辅助函数

In [7]:
def set_seed_everywhere(seed, cuda):
    np.random.seed(seed)
    torch.manual_seed(seed)
    if cuda:
        torch.cuda.manual_seed_all(seed)

def handle_dirs(dirpath):
    if not os.path.exists(dirpath):
        os.makedirs(dirpath)

## 超参数

In [8]:
try:
    from google.colab import drive
    # drive.mount('/content/folders/')
    json_train_path = '/content/folders/My Drive/HotpotQA/hotpot_train_v1.1.json'
    save_cache_path = '/content/folders/My Drive/save_cache'
    HotpotQA_path = '/content/folders/My Drive/HotpotQA'
except:
    json_train_path = 'data/hotpot_train_v1.1.json'
    save_cache_path = 'save_cache/'
    HotpotQA_path = './'

args = Namespace(
    # Data and path information
    json_train_path=json_train_path,
    model_state_file = "model_HotpotQA.pth",
    save_dir = save_cache_path,
    HotpotQA_preprocess_file = 'hotpotQA_train_preprocess100.pkl',

    # Model hyper parameter
    use_proxy = False,
    proxies={"http_proxy": "127.0.0.1:10802",
             "https_proxy": "127.0.0.1:10802"} if use_proxy else None,
    tokenizer_type = "bert-large-cased-whole-word-masking",
    model_type = 'bert-base-uncased',

    # Dataset parameter
    max_seq = 512,
    pad_to_max = True,

    # Training hyper parameter
    num_epochs=2,
    learning_rate=1e-3,
    batch_size=64,
    seed=1337,
    early_stopping_criteria=5,
    freeze_layer_name = 'bert.encoder.layer.10',

    # Runtime hyper parameter
    cuda=True,
    device=None,
    tpu=False,
    catch_keyboard_interrupt=True,
    reload_from_files=True,
    expand_filepaths_to_save_dir=True,
)


## 参数配置

In [9]:
# Check TPU
if args.tpu:
    try:
        import torch_xla
        import torch_xla.core.xla_model as xm
        args.device = xm.xla_device()
    except:
        # 安装完可能要重启内核
        VERSION = "20200220"
        _=!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
        _=!python pytorch-xla-env-setup.py --version $VERSION
        import torch_xla
        import torch_xla.core.xla_model as xm
        args.device = xm.xla_device()

# Check CUDA
if not torch.cuda.is_available():
    args.cuda = False

if not args.device:
    args.device = torch.device("cuda" if args.cuda else "cpu")
    
print("Using: {}".format(args.device))


Using: cuda


In [10]:
if args.expand_filepaths_to_save_dir:
    args.model_state_file = os.path.join(args.save_dir,args.model_state_file)
    args.HotpotQA_preprocess_file = os.path.join(args.save_dir,args.HotpotQA_preprocess_file)

# Set seed for reproducibility
set_seed_everywhere(args.seed, args.cuda)

# handle dirs
handle_dirs(args.save_dir)

# init pre-trained model
tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_type,proxies=proxies)
classifier = AutoModelForSequenceClassification.from_pretrained(args.model_type,proxies=proxies)

if args.reload_from_files:
    dataset = HotpotQA_Dataset.build_dataset_from_path(args.HotpotQA_preprocess_file, tokenizer=tokenizer)
else:
    dataset = HotpotQA_Dataset.build_dataset(hotpotQA_train_preprocess, tokenizer=tokenizer)

dataset.set_parameters(args.max_seq,args.pad_to_max)

print(dataset)

HotpotQA Dataset. mode: train. pair: ques_sent. size: 312. max_seq: 512


## 训练流程

### 训练辅助函数

In [11]:
def make_train_state(args):
    return {'stop_early': False,
            'early_stopping_step': 0,
            'early_stopping_best_val': 1e8,
            'learning_rate': args.learning_rate,
            'epoch_index': 0,
            'train_loss': [],
            'train_acc': [],
            'val_loss': [],
            'val_acc': [],
            'test_loss': -1,
            'test_acc': -1,
            'model_filename': args.model_state_file}


def update_train_state(args, model, train_state):
    # Save one model at least
    if train_state['epoch_index'] == 0:
        torch.save(model.state_dict(), train_state['model_filename'])
        train_state['stop_early'] = False

    # Save model if performance improved
    elif train_state['epoch_index'] >= 1:
        loss_tm1, loss_t = train_state['val_loss'][-2:]
         
        # If loss worsened
        if loss_t >= loss_tm1:
            # Update step
            train_state['early_stopping_step'] += 1
        # Loss decreased
        else:
            # Save the best model
            if loss_t < train_state['early_stopping_best_val']:
                torch.save(model.state_dict(), train_state['model_filename'])
                train_state['early_stopping_best_val'] = loss_t

            # Reset early stopping step
            train_state['early_stopping_step'] = 0

        # Stop early ?
        train_state['stop_early'] = \
            train_state['early_stopping_step'] >= args.early_stopping_criteria

    return train_state

def compute_accuracy(logits, y_target):
    _, logits_indices = logits.max(dim=1)
    n_correct = torch.eq(logits_indices, y_target).sum().item()
    return n_correct / len(logits_indices) * 100

# 冻结
def freeze_to_layer(model, layer_name):
    '''冻结层. 从0到layer_name.'''
    index_start = -1
    for index, (key, _value) in enumerate(model.state_dict().items()):
        if key.startswith(layer_name): 
            index_start = index
            break
    
    if index_start < 0:
        print(f"Don't find layer name: {layer_name}")
        return
    
    no_grad_nums = index_start + 1
    grad_nums = 0

    for index, i in enumerate(model.parameters()):
        if index >= index_start:
            i.requires_grad = True
            grad_nums += 1
        else:
            i.requires_grad = False
    
    print(f"freeze layers num: {no_grad_nums}, active layers num: {grad_nums}.")

### 实例化

In [12]:
classifier = classifier.to(args.device)
classifier.train()
freeze_to_layer(classifier, args.freeze_layer_name)
# dataset.class_weights = dataset.class_weights.to(args.device)
    
# loss_func = nn.CrossEntropyLoss(dataset.class_weights)
loss_func = nn.CrossEntropyLoss()

optimizer = optim.Adam(filter(lambda p: p.requires_grad, classifier.parameters()),
                      lr=args.learning_rate)

classifier, optimizer = amp.initialize(classifier, optimizer)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,\
                        mode='min', factor=0.5, patience=1)

train_state = make_train_state(args)

epoch_bar = tqdm_notebook(desc='training routine',
                total=args.num_epochs,
                position=0)

dataset.set_split('train')
train_bar = tqdm_notebook(desc='split=train',
                total=dataset.get_num_batches(args.batch_size), 
                position=1, 
                leave=True)
dataset.set_split('val')
val_bar = tqdm_notebook(desc='split=val',
                total=dataset.get_num_batches(args.batch_size), 
                position=1, 
                leave=True)

freeze layers num: 166, active layers num: 36.
Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


### 训练

In [13]:
try:
    for epoch_index in range(args.num_epochs):
        train_state['epoch_index'] = epoch_index
        # print('F1')
        # Iterate over training dataset

        # setup: batch generator, set loss and acc to 0, set train mode on
        dataset.set_split('train')
        batch_generator = generate_batches(dataset,
                            batch_size=args.batch_size, 
                            device=args.device)
        running_loss = 0.0
        running_acc = 0.0
        classifier.train()

        for batch_index, batch_dict in enumerate(batch_generator):
            # the training routine is these 5 steps:

            # --------------------------------------    
            # step 1. zero the gradients
            optimizer.zero_grad()

            # step 2. compute the output
            y_target = batch_dict.pop('labels')
            # print(batch_dict)
            res_tuple = classifier(**batch_dict)
            logits = res_tuple[0]
#             if type(logits) != torch.Tensor: print(logits)

            # step 3. compute the loss

            loss = loss_func(logits, y_target)

            # 移动平均. 递归数列可推导通项公式,求的是整个batch的均值.
            running_loss += (loss.item() - running_loss) / (batch_index + 1)

            # step 4. use loss to produce gradients
            with amp.scale_loss(loss, optimizer) as scaled_loss:
                scaled_loss.backward()

            # step 5. use optimizer to take gradient step
            optimizer.step()
            # -----------------------------------------
            # compute the accuracy
            acc_t = compute_accuracy(logits, y_target)
            running_acc += (acc_t - running_acc) / (batch_index + 1)

            # update bar
            train_bar.set_postfix(loss=running_loss, acc=running_acc, epoch=epoch_index)
            train_bar.update()

        train_state['train_loss'].append(running_loss)
        train_state['train_acc'].append(running_acc)

        # Iterate over val dataset

        # setup: batch generator, set loss and acc to 0; set eval mode on

        dataset.set_split('val')
        batch_generator = generate_batches(dataset, 
                            batch_size=args.batch_size, 
                            device=args.device)
        running_loss = 0.
        running_acc = 0.
        classifier.eval()

        for batch_index, batch_dict in enumerate(batch_generator):
            # compute the output
            with torch.no_grad():
                y_target = batch_dict.pop('labels')
                res_tuple = classifier(**batch_dict)
                logits = res_tuple[0]

                # step 3. compute the loss
                loss = loss_func(logits, y_target)
                running_loss += (loss.item() - running_loss) / (batch_index + 1)

                # compute the accuracy
                acc_t = compute_accuracy(logits, y_target)
                running_acc += (acc_t - running_acc) / (batch_index + 1)
                
            val_bar.set_postfix(loss=running_loss, acc=running_acc, epoch=epoch_index)
            val_bar.update()

        train_state['val_loss'].append(running_loss)
        train_state['val_acc'].append(running_acc)

        train_state = update_train_state(args=args, model=classifier, train_state=train_state)

        scheduler.step(train_state['val_loss'][-1])

        train_bar.n = 0
        val_bar.n = 0
        epoch_bar.update()

        if train_state['stop_early']:
            break
            
except KeyboardInterrupt:
    print("Exiting loop")

# 草稿

In [ ]:
tokenizer_BERT = AutoTokenizer.from_pretrained("bert-large-cased-whole-word-masking")

In [ ]:
with open(f'{save_cache_path}/hotpotQA_train_preprocess100.pkl','rb')as fp:
    hotpotQA_train_preprocess = pickle.load(fp)
len(hotpotQA_train_preprocess)

In [ ]:
hotpotQA_train_preprocess[0].keys()

In [ ]:
test_dataset = HotpotQA_Dataset.build_dataset(hotpotQA_train_preprocess,tokenizer=tokenizer_BERT)
test_dataset

In [ ]:
dataset

In [ ]:
test_dataset.set_parameters(512,True)

In [ ]:
def generate_batches(dataset, batch_size, shuffle=True, drop_last=True, device='cpu'): 
    dataloader = DataLoader(dataset=dataset, batch_size=batch_size,
                    shuffle=shuffle, drop_last=drop_last)

    for data_dict in dataloader:
        out_data_dict = {}
        for name, _tensor in data_dict.items():
            out_data_dict[name] = data_dict[name].to(device).squeeze(1)
        yield out_data_dict

for i in generate_batches(dataset, 3, device=args.device):
    test_one_batch = i
    print(i)
    for k,v in i.items(): print(v.shape)
    break

In [ ]:
test_item = test_dataset.get_one_item(2)
print(test_item)

In [ ]:
test_one_input = test_dataset.__getitem__(2)
# print(test_one_input)

In [ ]:
from transformers import BertForSequenceClassification, BertConfig
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
# 自行从头开始训练BERT..
my_config = BertConfig.from_pretrained('bert-base-uncased',
                    max_position_embeddings = 1024)

In [ ]:
# load预训练模型 position embedding只有512.
model_BERT_pre = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased')
_ = model_BERT_pre.to(args.device).train()

In [ ]:
model_BERT_pre(**test_one_batch)
# model_loss, model_logits

In [ ]:
test_one_batch.pop('labels')

In [ ]:
for k,v in test_one_batch.items():
    print(k,v.shape)

In [ ]:
model_BERT_pre(**test_one_batch)
# model_loss, model_logits

In [ ]:
test_one_batch